<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/nagoya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インストール

In [0]:
!apt install poppler-utils poppler-data

In [0]:
!wget https://www-eu.apache.org/dist/pdfbox/2.0.17/pdfbox-app-2.0.17.jar -O pdfbox-app.jar

# ループ

PDFファイルを選ぶ以外は共通

## ファイルをクリア

In [0]:
!rm *.pdf
!rm *.xml
!rm *.csv

!rm result.tsv

## PDFファイルを選ぶ

In [0]:
# kaitodeta1.pdf
!wget http://www.city.nagoya.jp/kenkofukushi/cmsfiles/contents/0000088/88972/kaitodeta1.pdf -O data.pdf

!java -jar pdfbox-app.jar PDFSplit -split 155 data.pdf

!for i in {1..16}; do pdftohtml -q -xml data-${i}.pdf data-${i}.xml; done

n = 17

pdf_num = 1

In [0]:
# kaitodeta2.pdf
!wget http://www.city.nagoya.jp/kenkofukushi/cmsfiles/contents/0000088/88972/kaitodeta2.pdf -O data.pdf

!java -jar pdfbox-app.jar PDFSplit -split 155 data.pdf

!for i in {1..16}; do pdftohtml -q -xml data-${i}.pdf data-${i}.xml; done

n = 17

pdf_num = 2

In [0]:
# kaitodeta3.pdf
!wget http://www.city.nagoya.jp/kenkofukushi/cmsfiles/contents/0000088/88972/kaitodeta3.pdf -O data.pdf

!java -jar pdfbox-app.jar PDFSplit -split 155 data.pdf

!for i in {1..25}; do pdftohtml -q -xml data-${i}.pdf data-${i}.xml; done

n = 26

pdf_num = 3

In [0]:
# kaitodeta4.pdf
!wget http://www.city.nagoya.jp/kenkofukushi/cmsfiles/contents/0000088/88972/kaitodeta4.pdf -O data.pdf

!java -jar pdfbox-app.jar PDFSplit -split 155 data.pdf

!for i in {1..17}; do pdftohtml -q -xml data-${i}.pdf data-${i}.xml; done

n = 18

pdf_num = 4

In [0]:
# kaitodeta5.pdf
!wget http://www.city.nagoya.jp/kenkofukushi/cmsfiles/contents/0000088/88972/kaitodeta5.pdf -O data.pdf

!java -jar pdfbox-app.jar PDFSplit -split 175 data.pdf

!for i in {1..16}; do pdftohtml -q -xml data-${i}.pdf data-${i}.xml; done

n = 17

pdf_num = 5

## XMLからCSVに変換

In [0]:
from xml.etree import ElementTree as ET
import csv

result = []

for i in range(1, n):

    tree = ET.parse(f"data-{i}.xml")
    root = tree.getroot()

    pages = root.findall("page")

    for page in pages:

        # print(page.attrib)

        for item in page.findall("text"):

            temp = item.attrib
            temp["side"] = i
            temp["text"] = item.text.strip()
            temp["page"] = page.get("number")

            if temp["text"]:
                result.append(temp)
            else:
                print(item.text)

with open("data.csv", "w", newline="", encoding="utf-8") as fw:

    fieldnames = ["page", "side", "top", "left", "width", "height", "font", "text"]

    writer = csv.DictWriter(fw, fieldnames=fieldnames)

    writer.writeheader()

    for row in result:
        writer.writerow(row)

## 前処理

In [0]:
import pandas as pd

df1 = pd.read_csv("data.csv")

# 1ページ目のヘッダー部分を削除
df2 = df1.drop(df1[(df1["page"] == 1) & (df1["top"] < 510)].index)

# ページ・サイド・縦・横・テキスト抽出
df3 = df2.loc[:, ["page", "side", "top", "left", "text"]]

In [0]:
df3

### 対象のPDFファイルのずれ補正を実行

In [0]:
# kaitodeta1.pdf
if pdf_num == 1:

    df3.loc[(df3["side"] == 8) & (df3["left"] == 383), "left"] = 382
    df3.loc[(df3["side"] == 8) & (df3["left"] == 765), "left"] = 766
    df3.loc[(df3["side"] == 12) & (df3["left"] == 646), "left"] = 647
    df3.loc[(df3["side"] == 12) & (df3["left"] == 882), "left"] = 883
    df3.loc[(df3["side"] == 14) & (df3["left"] == 648), "left"] = 647
    df3.loc[(df3["side"] == 14) & (df3["left"] == 680), "left"] = 681

    # 220列に空列生成
    s1 = pd.Series([1, 10, 522, 600, ""], index=df3.columns, name=9999998)
    df3 = df3.append(s1)

    # 223列に空列生成
    s2 = pd.Series([1, 10, 522, 800, ""], index=df3.columns, name=9999999)
    df3 = df3.append(s2)

# kaitodeta2.pdf
elif pdf_num == 2:
    df3.loc[(df3["side"] ==9) & (df3["left"] == 383), "left"] = 382
    df3.loc[(df3["side"] ==12) & (df3["left"] == 356), "left"] = 355
    df3.loc[(df3["side"] ==12) & (df3["left"] == 857), "left"] = 856
    df3.loc[(df3["side"] ==12) & (df3["left"] == 1125), "left"] = 1126
    df3.loc[(df3["side"] ==13) & (df3["left"] == 560), "left"] = 559
    df3.loc[(df3["side"] ==13) & (df3["left"] == 592), "left"] = 593

    # 220列に空列生成
    s1 = pd.Series([1, 9, 519, 900, ""], index=df3.columns, name=9999999)
    df3 = df3.append(s1)

# kaitodeta3.pdf
elif pdf_num == 3:
    df3.loc[(df3["side"] ==24) & (df3["left"] == 739), "left"] = 740
    df3.loc[(df3["side"] ==24) & (df3["left"] == 852), "left"] = 851
    df3.loc[(df3["side"] ==24) & (df3["left"] == 884), "left"] = 885


# kaitodeta4.pdf
elif pdf_num == 4:
    df3.loc[(df3["side"] ==6) & (df3["left"] == 682), "left"] = 683
    df3.loc[(df3["side"] ==15) & (df3["left"] == 651), "left"] = 652

# kaitodeta5.pdf
elif pdf_num == 5:
    df3.loc[(df3["side"] ==8) & (df3["left"] == 767), "left"] = 768
    df3.loc[(df3["side"] ==11) & (df3["left"] == 356), "left"] = 355
    df3.loc[(df3["side"] ==13) & (df3["left"] == 1027), "left"] = 1026
    df3.loc[(df3["side"] ==13) & (df3["left"] == 1060), "left"] = 1059
    df3.loc[(df3["side"] ==15) & (df3["left"] == 470), "left"] = 469
    df3.loc[(df3["side"] ==15) & (df3["left"] == 502), "left"] = 503

### ずれ確認

In [0]:
df_count = df3.pivot_table(index=["side", "left"], aggfunc="count")
df_count

In [0]:
# df_count.to_csv('count.tsv', sep='\t')

### 並び替え

In [0]:
# ページ
df = df3.pivot_table(index=["page", "top"], columns=["side", "left"], values="text", aggfunc=lambda x: ' '.join(str(v) for v in x))

In [0]:
df

### TSV変換

In [0]:
df.to_csv('result.tsv', sep='\t', index=False, header=False)

## TSV調整

In [0]:
import re

with open("result.tsv") as fr:
    tsv_text = fr.read()

pattern1 = re.compile(r"(?<=\d) (?=\d)")
tsv_text = pattern1.sub("\t", tsv_text)

pattern2 = re.compile(r" (\d)\t\t")
tsv_text = pattern2.sub(r"\t\1\t", tsv_text)


if pdf_num == 1:
    tsv_text = re.sub(r"0\t\[04\] 月経量の異常による低血圧\t0\t\t00", r"0\t[04]\t月経量の異常による低血圧\t0\t00", tsv_text)

with open(f"result-{pdf_num}.tsv", mode="w") as fw:
    fw.write(tsv_text)

# TSV結合

In [0]:
dfs = []

for i in range(1, 6):
    dfs.append(pd.read_csv(f"result-{i}.tsv", delimiter='\t', header=None, dtype=object))

df4 = pd.concat(dfs)

In [101]:
df4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277
0,000001,0,0,1,0,0,0,1,0,0,0,0,0000,0,0,0,0,0,2,0000,0,0,1,0,0,0,0000,0,0,0,0,0,0,0000,0,0,0,0,0,0,...,0,2,1301,0,1302,0,1404,0,2,1008,0,2,2209,0,2,1105,0,2,1208,0,2,1912,0,0,・おたふく ・水痘,2,1306,0,0,0000,0,2,0,0,0,0,NaN,NaN,NaN,NaN
1,000002,0,1,0,0,0,0,0,0,1,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,1504,0,1505,0,2308,0,2,2307,0,2,2411,0,2,1305,0,2,1311,0,2,2610,0,1,NaN,0,0000,0,0,0000,0,2,0,0,0,0,NaN,NaN,NaN,NaN
2,000003,0,1,0,0,1,0,0,0,0,0,2,0000,2,0,1,0,0,1,0000,0,0,0,0,0,2,0000,2,0,0,0,0,1,0000,0,0,0,0,0,2,...,0,2,1105,0,1106,0,1208,0,2,0805,0,2,2008,0,2,0905,0,2,0906,0,2,0000,0,1,おたふくかぜ,0,1504,0,0,0000,0,1,0,0,0,0,NaN,NaN,NaN,NaN
3,000004,0,0,1,0,0,0,0,0,0,1,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,0,0000,0,0000,0,0000,0,0,0000,0,0,0000,0,0,0000,0,0,0000,0,2,2511,0,0,NaN,0,0000,0,0,0000,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,000005,0,0,1,0,0,0,0,1,0,0,2,2610,1,1,0,0,0,2,2610,1,1,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,1403,0,1601,0,2309,0,2,1210,0,2,2307,0,2,1208,0,2,1302,0,2,0000,0,1,NaN,0,2611,0,0,0000,0,1,0,0,0,0,NaN,NaN,NaN,NaN
5,000006,1,0,0,0,1,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,2309,0,2311,0,0000,0,1,0000,0,2,2101,0,1,0000,0,1,0000,0,1,0000,0,0,NaN,1,0000,0,1,0000,0,2,0,0,0,0,NaN,NaN,NaN,NaN
6,000007,0,1,0,0,0,0,1,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,0000,0,2109,0,0000,0,2,2109,0,2,2207,0,0,0000,0,0,0000,0,2,2612,0,1,NaN,1,0000,0,1,0000,0,1,0,0,0,0,NaN,NaN,NaN,NaN
7,000008,1,0,0,0,0,0,1,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,1209,0,2309,0,2311,0,1,0000,0,2,2303,0,1,0000,0,1,0000,0,1,0000,0,0,NaN,1,0000,0,1,0000,0,2,0,0,0,0,NaN,NaN,NaN,NaN
8,000009,0,0,1,0,0,1,0,0,0,0,2,2210,1,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,2507,0,0,0,1,0,2,...,0,2,1000,0,1300,0,1400,0,2,1000,0,2,1200,0,2,1000,0,2,1500,0,2,1409,0,0,NaN,0,0000,0,0,0000,0,1,0,0,0,1,通知が来ませんでした。,NaN,NaN,NaN
9,000010,0,0,0,0,0,0,0,0,0,1,2,0000,2,0,1,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,0000,0,0,0,0,0,1,...,0,2,0000,0,0000,0,0000,0,2,0000,0,1,0000,0,1,0000,0,0,0000,0,2,2610,0,0,NaN,0,0000,0,0,0000,0,3,0,0,0,0,NaN,NaN,NaN,NaN


In [0]:
# 275質５＿２中止理由その他を結合
df4[274] = df4[274].fillna("") + df4[275].fillna("") + df4[276].fillna("") + df4[277].fillna("")

In [0]:
# 275質５＿２中止理由その他を不要部分を削除
df4.drop(columns=[275, 276, 277], inplace=True)

# 完成

In [0]:
# 最終結果をファイルに書き出し
df4.to_csv('result_all.csv', index=False, header=False)